In [1]:
# Get today date

# Extract hourly data from today and yesterday

# Convert hourly data into a dataframe

# Preprocess the dataframe
    # Differencing
    
    # Normalization (Use training_mean and training_std)

# Convert the dataset into a Window (Input width = 48)

# Load the AR model

# Predict future output

# Untransform data


# Remove the read_csv



In [2]:
# Dates
# import datetime as dt
from datetime import datetime, timedelta
import pytz

# REE API request
import requests
import json

# Convert extracted data into dataframe
import pandas as pd

# WindowGenerator
from window_generator import WindowGenerator
import numpy as np

# Loading the model
import tensorflow as tf
from tensorflow import keras

# Extract data in JSON format

In [3]:
def extract_json_data(day):
    URLprices = "https://apidatos.ree.es/es/datos/mercados/precios-mercados-tiempo-real"
    URLdemand = "https://apidatos.ree.es/es/datos/demanda/demanda-tiempo-real"
    URLEmissions = "https://apidatos.ree.es/es/datos/generacion/evolucion-estructura-generacion-emisiones-asociadas"
    
    startHour = "00:00"
    endHour = "23:59"
    
    date_start = str(day.year) + "-" + str(day.month) + "-" + str(day.day) + "T" + startHour
    date_end = str(day.year) + "-" + str(day.month) + "-" + str(day.day) + "T" + endHour
    
    PARAMS = {'start_date':date_start, 'end_date':date_end,"time_trunc":"hour", 'geo_limit': 'peninsular'}
    
    prices = requests.get(url=URLprices, params=PARAMS)
    demands = requests.get(url=URLdemand, params=PARAMS)
    emissions = requests.get(url=URLEmissions, params=PARAMS)
    
    pricesJ = json.dumps(prices.json(), indent=4)
    demandJ = json.dumps(demands.json(), indent=4)
    emissionsJ = json.dumps(emissions.json(), indent=4)
    
    return pricesJ, demandJ, emissionsJ
    

In [4]:
timezone = pytz.timezone('Europe/Madrid')

today = datetime.now(tz=timezone)
yesterday = today - timedelta(days=1)
day_zero = yesterday - timedelta(days=1)


In [5]:
pricesJ_today, demandJ_today, emissionsJ_today = extract_json_data(today)
pricesJ_yesterday, demandJ_yesterday, emissionsJ_yesterday = extract_json_data(yesterday)
pricesJ_day_zero, demandJ_day_zero, emissionsJ_day_zero = extract_json_data(day_zero)

# Convert json data into a dataframe

In [6]:
type(pricesJ_day_zero)

str

In [7]:
def clean_datetime(timestamp):
    r = timestamp.replace(".000+01:00", "")
    r = r.replace(".000+02:00", "")
    return r

In [8]:
def convert_json_into_dataframe(price_json, demand_json, emissions_json, today = False):
    
    df_price = pd.DataFrame(columns=['timestamp', 'price'])
    df_demand = pd.DataFrame(columns=['timestamp', 'demand'])
    df_emissions = pd.DataFrame(columns=['timestamp', 'emissions'])
    
    # Generate price dataframe
    price_json = json.loads(price_json)
    prices_values = price_json['included'][0]['attributes']['values']
    for price_value in prices_values:
        row = {'timestamp': price_value['datetime'],
                   'price': price_value['value']}
        df_price = df_price.append(row, ignore_index=True)
        
    # Generate demand dataframe
    demand_json = json.loads(demand_json)
    if(today):
        demand_values = demand_json['included'][1]['attributes']['values']
    else:
        demand_values = demand_json['included'][0]['attributes']['values']

    for demand_value in demand_values:
        row = {'timestamp': demand_value['datetime'],
                   'demand': demand_value['value']}
        df_demand = df_demand.append(row, ignore_index=True)
    
    # Generate emissions dataframe
    emissions_json = json.loads(emissions_json)
    emissions_values = emissions_json['included'][0]['attributes']['values']

    for emissions_value in emissions_values:
        row = {'timestamp': emissions_value['datetime'],
                'emissions': emissions_value['value']}
        df_emissions = df_emissions.append(row, ignore_index=True)    
    
    df_merged = df_price.merge(df_demand, on='timestamp', how='inner')
    
    df_emissions['timestamp'] = df_price['timestamp']
    df_merged = df_merged.merge(df_emissions, on='timestamp', how='inner')
    
    df_merged['timestamp'] = df_merged['timestamp'].apply(clean_datetime)
    
    # Separate two columns by " "
    df_merged[['date', 'time']] = df_merged['timestamp'].str.split('T', expand=True)
    
    df_merged = df_merged.drop(columns=['timestamp'], axis=1)
    
    return df_merged

In [9]:
# day_zero
df_final = convert_json_into_dataframe(pricesJ_day_zero, demandJ_day_zero, emissionsJ_day_zero)

# yesterday
df_yesterday = convert_json_into_dataframe(pricesJ_yesterday, demandJ_yesterday, emissionsJ_day_zero)
# Repetimos las emisiones del día cero
df_final = pd.concat([df_final, df_yesterday], ignore_index=True)


# today
df_today = convert_json_into_dataframe(pricesJ_today, demandJ_today, emissionsJ_day_zero, today=True)
df_final = pd.concat([df_final, df_today], ignore_index=True)


In [10]:
# df_final.to_csv('data.csv', index=False)
df = df_final

In [11]:
del df_final

# Change the index

In [12]:
df['data'] = df['date'] + " " + df['time']

In [13]:
df.head()

,price,demand,emissions,date,time,data
0,254.30,26792,12287.250,2022-02-11,00:00:00,2022-02-11 00:00:00
1,233.71,25817,11883.631,2022-02-11,01:00:00,2022-02-11 01:00:00
2,232.08,24690,10940.722,2022-02-11,02:00:00,2022-02-11 02:00:00
3,222.96,24232,11452.998,2022-02-11,03:00:00,2022-02-11 03:00:00
4,219.90,23537,12749.593,2022-02-11,04:00:00,2022-02-11 04:00:00


In [14]:
df['data'] = pd.to_datetime(df['data'])
df = df.drop(columns=['date', 'time'], axis=1)
df = df.set_index('data')

In [15]:
df.head()

,price,demand,emissions
data,,,
2022-02-11 00:00:00,254.30,26792,12287.250
2022-02-11 01:00:00,233.71,25817,11883.631
2022-02-11 02:00:00,232.08,24690,10940.722
2022-02-11 03:00:00,222.96,24232,11452.998
2022-02-11 04:00:00,219.90,23537,12749.593


In [16]:
# Used for removing differencing
last_line = df.tail(n=1)
last_line

,price,demand,emissions
data,,,
2022-02-13 23:00:00,219.59,25770,12672.25


In [17]:
initial_price = last_line['price'][0]
initial_demand = last_line['demand'][0]
initial_emissions = last_line['emissions'][0]

# Preprocessing

In [18]:
# Normalization/Standarization params
train_mean = pd.Series(data = [0.025869, -0.074058, 0.272317], index = ['price', 'demand', 'emissions'])
train_std = pd.Series(data = [22.190049, 1295.266304, 537.640099], index = ['price', 'demand', 'emissions'])

## Differencing

In [19]:
def difference(dataset, interval=1):
    diff = list()
    for i in range(interval, len(dataset)):
        value = dataset[i] - dataset[i - interval]
        diff.append(value)
    return pd.Series(diff)

In [20]:
diff_price = difference(df['price'])
diff_emissions = difference(df['emissions'])
diff_demand = difference(df['demand'])

In [21]:
s1 = pd.Series([0.0])
diff_price = s1.append(diff_price, ignore_index=True)
diff_emissions = s1.append(diff_emissions, ignore_index=True)
diff_demand = s1.append(diff_demand, ignore_index=True)

In [22]:
df['price'] = diff_price.values
df['demand'] = diff_demand.values
df['emissions'] = diff_emissions.values

In [23]:
df.head()

,price,demand,emissions
data,,,
2022-02-11 00:00:00,0.00,0.0,0.000
2022-02-11 01:00:00,-20.59,-975.0,-403.619
2022-02-11 02:00:00,-1.63,-1127.0,-942.909
2022-02-11 03:00:00,-9.12,-458.0,512.276
2022-02-11 04:00:00,-3.06,-695.0,1296.595


## Normalization

In [24]:
df = (df-train_mean)/train_std

## Select subset with length=input_size

In [25]:
df = df[len(df)-48:]

In [26]:
df.head()

,price,demand,emissions
data,,,
2022-02-12 00:00:00,-0.134559,-1.654429,-0.716599
2022-02-12 01:00:00,0.108793,-0.822168,-0.751230
2022-02-12 02:00:00,-0.149430,-1.178851,-1.754299
2022-02-12 03:00:00,-0.085888,-0.967311,0.952317
2022-02-12 04:00:00,-0.159795,-0.390596,2.411135


In [27]:
# Fill labels with zeros
L = 24
i = 0
while i<24:
    i = i + 1
    last_date = df.index[-1] + pd.Timedelta(hours=1)
    df.loc[last_date] = [0.0,0.0,0.0]

# Convert the dataframe into a window

In [28]:
OUT_STEPS = 24
multi_window = WindowGenerator(input_width=48, label_width=OUT_STEPS, shift=OUT_STEPS, train_df=df, test_df=df)
multi_window

Total window size: 72
Input indices: [ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23
 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47]
Label indices: [48 49 50 51 52 53 54 55 56 57 58 59 60 61 62 63 64 65 66 67 68 69 70 71]
Label column name(s): None

In [29]:
example_window = tf.stack([
    np.array(df)
])

example_inputs, example_labels = multi_window.split_window(example_window)

In [30]:
multi_window.example = example_inputs, example_labels

# Load model

In [31]:
ar_model = keras.models.load_model('saved_model/')

# Obtain predictions with the model

In [32]:
df.head()

,price,demand,emissions
data,,,
2022-02-12 00:00:00,-0.134559,-1.654429,-0.716599
2022-02-12 01:00:00,0.108793,-0.822168,-0.751230
2022-02-12 02:00:00,-0.149430,-1.178851,-1.754299
2022-02-12 03:00:00,-0.085888,-0.967311,0.952317
2022-02-12 04:00:00,-0.159795,-0.390596,2.411135


In [33]:
inputs, labels = multi_window.example
inputs

<tf.Tensor: shape=(1, 48, 3), dtype=float64, numpy=
array([[[-1.34558919e-01, -1.65442885e+00, -7.16598925e-01],
        [ 1.08793406e-01, -8.22167564e-01, -7.51229898e-01],
        [-1.49430450e-01, -1.17885097e+00, -1.75429868e+00],
        [-8.58884539e-02, -9.67311462e-01,  9.52316771e-01],
        [-1.59795456e-01, -3.90596081e-01,  2.41113467e+00],
        [-4.00443866e-01, -4.78094287e-02,  5.70522332e-01],
        [ 1.97121286e-01,  2.86227302e-02,  2.20668563e+00],
        [ 3.60257474e-01,  7.95260446e-01,  5.83548677e+00],
        [ 7.45438192e-02,  9.87498906e-01,  2.46967383e+00],
        [-2.00804829e-01,  1.37892420e+00,  3.65711874e+00],
        [-5.51863090e-01,  1.69468939e+00,  2.26696574e+00],
        [-1.00431815e+00,  3.09646022e-01,  9.33871346e-01],
        [-2.15225708e-01,  9.50183430e-02,  1.99975538e+00],
        [-5.07699149e-01, -2.86370410e-01,  6.85694446e-02],
        [-1.28700437e-01,  2.31669779e-01, -1.32152776e+00],
        [-1.01788419e-02, -9.5573

In [34]:
# Each column is a different df column
predictions = ar_model(inputs)

In [35]:
predictions

<tf.Tensor: shape=(1, 24, 3), dtype=float32, numpy=
array([[[ 0.1542171 , -0.336253  ,  0.4466094 ],
        [ 0.07333905, -0.19672896,  1.1792979 ],
        [-0.646824  , -0.01431297,  1.587493  ],
        [-1.2396584 ,  0.01039064,  0.97841537],
        [-0.40593883, -0.58786595, -0.00774271],
        [-0.47831804, -1.027991  , -0.5333189 ],
        [-0.21302146, -0.7915988 , -0.4719975 ],
        [-0.16622454, -0.6713635 , -0.2599334 ],
        [-0.13817772, -0.5350092 , -0.04583747],
        [-0.1395258 , -0.390204  ,  0.12825617],
        [-0.1311802 , -0.22928388,  0.24557075],
        [-0.09280051, -0.04908067,  0.2500883 ],
        [-0.03912248,  0.1692939 ,  0.08066442],
        [-0.06260327,  0.40725473, -0.20584056],
        [-0.22845563,  0.61041456, -0.4202956 ],
        [-0.19936875,  0.68375516, -0.45206714],
        [-0.02048293,  0.5892056 , -0.31930125],
        [-0.04908918,  0.42181715, -0.19507894],
        [-0.19711468,  0.16088697, -0.19492754],
        [-0.28415

# Untransform data

In [36]:
predictions[0]

<tf.Tensor: shape=(24, 3), dtype=float32, numpy=
array([[ 0.1542171 , -0.336253  ,  0.4466094 ],
       [ 0.07333905, -0.19672896,  1.1792979 ],
       [-0.646824  , -0.01431297,  1.587493  ],
       [-1.2396584 ,  0.01039064,  0.97841537],
       [-0.40593883, -0.58786595, -0.00774271],
       [-0.47831804, -1.027991  , -0.5333189 ],
       [-0.21302146, -0.7915988 , -0.4719975 ],
       [-0.16622454, -0.6713635 , -0.2599334 ],
       [-0.13817772, -0.5350092 , -0.04583747],
       [-0.1395258 , -0.390204  ,  0.12825617],
       [-0.1311802 , -0.22928388,  0.24557075],
       [-0.09280051, -0.04908067,  0.2500883 ],
       [-0.03912248,  0.1692939 ,  0.08066442],
       [-0.06260327,  0.40725473, -0.20584056],
       [-0.22845563,  0.61041456, -0.4202956 ],
       [-0.19936875,  0.68375516, -0.45206714],
       [-0.02048293,  0.5892056 , -0.31930125],
       [-0.04908918,  0.42181715, -0.19507894],
       [-0.19711468,  0.16088697, -0.19492754],
       [-0.28415918, -0.20190978, -0.27

In [37]:
price_predictions = np.array(predictions[0,:,0])
demand_predictions = np.array(predictions[0,:,1])
emissions_predictions = np.array(predictions[0,:,2])

In [38]:
df.head()

,price,demand,emissions
data,,,
2022-02-12 00:00:00,-0.134559,-1.654429,-0.716599
2022-02-12 01:00:00,0.108793,-0.822168,-0.751230
2022-02-12 02:00:00,-0.149430,-1.178851,-1.754299
2022-02-12 03:00:00,-0.085888,-0.967311,0.952317
2022-02-12 04:00:00,-0.159795,-0.390596,2.411135


## Remove normalization

In [39]:
# Normalization
price_predictions = (price_predictions * train_std['price']) + train_mean['price']
demand_predictions = (demand_predictions * train_std['demand']) + train_mean['demand']
emissions_predictions = (emissions_predictions * train_std['emissions']) + train_mean['emissions']

## Remove differencing

In [40]:
price_predictions[0] = initial_price + price_predictions[0]
demand_predictions[0] = initial_demand + demand_predictions[0]
emissions_predictions[0] = initial_emissions + emissions_predictions[0]

for i in range(1, len(price_predictions)):
    price_predictions[i] = price_predictions[i-1] + price_predictions[i]
    demand_predictions[i] = demand_predictions[i-1] + demand_predictions[i]
    emissions_predictions[i] = emissions_predictions[i-1] + emissions_predictions[i]
    

In [41]:
price_predictions

array([223.03795 , 224.69121 , 210.36401 , 182.8818  , 173.89987 ,
       163.31184 , 158.61076 , 154.9481  , 151.9078  , 148.83759 ,
       145.95256 , 143.91917 , 143.07692 , 141.71362 , 136.67004 ,
       132.27191 , 131.84326 , 130.77985 , 126.43173 , 120.15209 ,
       116.5444  , 115.42854 , 119.658066, 129.69098 ], dtype=float32)

In [42]:
demand_predictions

array([25334.389, 25079.498, 25060.885, 25074.27 , 24312.752, 22981.156,
       21955.75 , 21086.082, 20393.03 , 19887.537, 19590.479, 19526.832,
       19746.04 , 20273.469, 21064.045, 21949.615, 22712.719, 23259.01 ,
       23467.328, 23205.727, 22569.8  , 21904.271, 21532.098, 21447.617],
      dtype=float32)

In [43]:
emissions_predictions

array([12912.638, 13546.948, 14400.721, 14927.028, 14923.138, 14636.677,
       14383.185, 14243.706, 14219.334, 14288.562, 14420.862, 14555.592,
       14599.232, 14488.837, 14263.142, 14020.364, 13848.968, 13744.357,
       13639.829, 13490.423, 13331.214, 13267.054, 13379.344, 13682.222],
      dtype=float32)